# 0. Введение

### Анализ и прогнозирование цен на недвижимость играют важную роль в различных областях. В данном исследовании рассматривается создание модели эластичной сети для предсказания цен на дома. Модель оценивается на основе различных параметров, демонстрируя высокую эффективность в прогнозировании цен.

# 1. Импорт базовых библиотек

In [108]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 2. Импорт файла с данными о домах

In [109]:
file_path = r'E:\Портфолио\Предсказание цен домов\Final_df.csv'
df = pd.read_csv(file_path)
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


# 3. Создание модели

## 3.1 Разделение данных

In [110]:
from sklearn.model_selection import train_test_split

# разделение данных
target_var = 'SalePrice'
X = df.drop(target_var, axis=1)
y = df[target_var]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

## 3.2 Создание модели ElasticNet

In [111]:
# вывод всех параметров, что находятся в модели
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet

scaler = StandardScaler()
elastic = ElasticNet(max_iter=100_000_000)

elastic.get_params().keys()

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'l1_ratio', 'max_iter', 'positive', 'precompute', 'random_state', 'selection', 'tol', 'warm_start'])

In [112]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Задаем сетку параметров для поиска
param_grid = {
    'elasticnet__alpha': [1, 10, 30, 50, 70, 80, 90, 100],
    'elasticnet__l1_ratio': np.arange(0.1, 1.1, 0.1)
}

# Создаем пайплайн с использованием ElasticNet
elastic_pipeline = Pipeline([
    ('scaler', scaler),
    ('elasticnet', elastic)
])

In [113]:
# Создаем объект GridSearchCV
grid_model_elastic = GridSearchCV(estimator=elastic_pipeline,
                                  param_grid=param_grid,
                                  scoring='neg_mean_squared_error',
                                  cv=5,
                                  verbose=1)

# Обучаем модель на обучающих данных
grid_model_elastic.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('elasticnet',
                                        ElasticNet(max_iter=100000000))]),
             param_grid={'elasticnet__alpha': [1, 10, 30, 50, 70, 80, 90, 100],
                         'elasticnet__l1_ratio': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             scoring='neg_mean_squared_error', verbose=1)

In [114]:
# Выводим лучшие параметры и результаты поиска
print("Best Parameters (Elastic Net): ", grid_model_elastic.best_params_)

Best Parameters (Elastic Net):  {'elasticnet__alpha': 100, 'elasticnet__l1_ratio': 1.0}


# 4. Анализ работы модели

## 4.1 Функция для оценки работы модели

In [115]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def analysis_metric(model, X_test, y_test):
    
    y_pred = model.predict(X_test)
    mean = y_test.mean()
    MAE = mean_absolute_error(y_test, y_pred)
    MSE = mean_squared_error(y_test, y_pred)
    RMSE = np.sqrt(MSE)
    r2 = round(r2_score(y_test, y_pred), 4)
    
    print('-------------')
    print('АНАЛИЗ МЕТРИК')
    print('-------------')
    
    print(f'mean: {round(mean, 4)}')
    print(f'MAE: {round(MAE, 4)}')
    print(f'MSE: {round(MSE, 4)}')
    print(f'RMSE: {round(RMSE, 4)}')
    print('')

    print(f'MAE в %: {round(MAE/mean*100, 4)}')
    print(f'RMSE в %: {round(RMSE/mean*100, 4)}')
    print(f'R-квадрат: {r2}' )

In [116]:
analysis_metric(grid_model_elastic, X_test=X_test, y_test=y_test)

-------------
АНАЛИЗ МЕТРИК
-------------
mean: 184537.3515
MAE: 14195.3549
MSE: 422652274.495
RMSE: 20558.5086

MAE в %: 7.6924
RMSE в %: 11.1406
R-квадрат: 0.919


# 5. Обучение модели на всём наборе данных

In [117]:
# Создаем объект GridSearchCV
final_model = GridSearchCV(estimator=elastic_pipeline,
                                  param_grid=param_grid,
                                  scoring='neg_mean_squared_error',
                                  cv=5,
                                  verbose=1)

# Обучаем модель на обучающих данных
final_model.fit(X, y)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('elasticnet',
                                        ElasticNet(max_iter=100000000))]),
             param_grid={'elasticnet__alpha': [1, 10, 30, 50, 70, 80, 90, 100],
                         'elasticnet__l1_ratio': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             scoring='neg_mean_squared_error', verbose=1)

In [118]:
# Выводим лучшие параметры и результаты поиска
print("Best Parameters (Elastic Net): ", grid_model_elastic.best_params_)

Best Parameters (Elastic Net):  {'elasticnet__alpha': 100, 'elasticnet__l1_ratio': 1.0}


# 6. Сохранение модели

In [119]:
from joblib import dump
import os

folder_path = 'E:\\Портфолио\\Предсказание цен домов'
file_path = os.path.join(folder_path, 'elastic_model_house.joblib')

os.makedirs(folder_path, exist_ok=True)
dump(final_model, file_path)

print('Модель сохранена!')

Модель сохранена!


# 7. Импортирование модели

In [120]:
import joblib

# Указываем путь к папке и файлу
folder_path = 'E:\\Портфолио\\Предсказание цен домов\\'
file_name = 'elastic_model_house.joblib'
file_path = folder_path + file_name

# Загружаем модель из файла
model = joblib.load(file_path)
print('Модель успешно импортирована!')

Модель успешно импортирована!


# 8. Вывод 

### Успешно разработанная модель Elastic Net для прогнозирования цен на дома показала высокую эффективность, подтвержденную значительными результатами метрик. Средняя абсолютная ошибка (MAE) составила 14195.3549, что в процентном соотношении составляет 7.6924%, а корень из среднеквадратической ошибки (RMSE) равен 20558.5086, что составляет 11.1406% от средней цены. 

### Значение коэффициента детерминации (R-квадрат) оценивается как высокое – 0.919, что свидетельствует о хорошей соответствии модели данным. Таким образом, разработанная модель эластичной сети успешно предсказывает цены на дома, демонстрируя высокую точность и адекватность в контексте рассматриваемой задачи.